<a href="https://colab.research.google.com/github/kienvillegas/SnapSoil/blob/master/soil_prediction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd

# Paths to dataset and images
img_path = '/content/drive/MyDrive/Thesis_Dataset/soil_images'
csv_path = '/content/drive/MyDrive/Thesis_Dataset/sensor_data.csv'

# Load the data
df = pd.read_csv(csv_path)

# List of image files (only .jpg)
image_files = [f for f in os.listdir(img_path) if f.endswith(".jpg")]
image_files.sort()

# Add image filenames as 'id' in the dataframe
df['id'] = image_files

# Print initial dataset size and image count
num_rows = len(df)
num_images_before = len(image_files)
print(f"Number of rows before cleanup: {num_rows}")
print(f"Number of images before cleanup: {num_images_before}")

# Filter out rows with 0 or NaN values in N, P, K, or pH
df_invalid = df[df[['N', 'P', 'K', 'pH']].eq(0).any(axis=1) | df[['N', 'P', 'K', 'pH']].isna().any(axis=1)]

# Get image IDs of invalid records
image_ids_to_delete = df_invalid['id'].tolist()

# Delete corresponding images from the filesystem
for image_id in image_ids_to_delete:
    image_file_path = os.path.join(img_path, image_id)
    if os.path.exists(image_file_path):
        os.remove(image_file_path)
        print(f"Deleted image file: {image_file_path}")
    else:
        print(f"Image file not found: {image_file_path}")

# Now remove these invalid records from the dataframe
df = df[~df['id'].isin(image_ids_to_delete)]

# Save the updated DataFrame back to the CSV
df.to_csv(csv_path, index=False)


# Verify the number of images remaining
image_files_after = [f for f in os.listdir(img_path) if f.endswith(".jpg")]
num_images_after = len(image_files_after)

# Print results
print(f"Number of rows after cleanup: {len(df)}")
print(f"Number of images after cleanup: {num_images_after}")


ValueError: Length of values (710) does not match length of index (770)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))  # Adjust figure size if needed
for column in ['N', 'P', 'K', 'pH']:
    plt.subplot(2, 2, df.columns.get_loc(column) + 1)  # Create subplots
    sns.histplot(df[column], kde=True)  # Plot histogram with kernel density estimation
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')

plt.tight_layout()  # Adjust subplot spacing
plt.show()  # Display the plot

In [ ]:
import os
import numpy as np
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from sklearn.model_selection import train_test_split
import xgboost as xgb
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def create_cnn_model(input_shape):
  model = Sequential([
      Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
      MaxPooling2D((2,2)),
      Conv2D(64, (3,3), activation='relu',),
      MaxPooling2D((2,2)),
      Conv2D(128, (3,3), activation='relu'),
      GlobalAveragePooling2D()
  ])
  return model

def extract_features(img_path, model, preprocess_input):
  img = image.load_img(img_path, target_size=(224,224))
  img_data = image.img_to_array(img)
  img_data = np.expand_dims(img_data, axis=0)
  if preprocess_input :
    img_data = preprocess_input(img_data)
  else :
    img_data = img_data / 255.0

  # datagen = ImageDataGenerator(
  #     rotation_range = 20,
  #     width_shift_range = 0.2,
  #     height_shift_range = 0.2,
  #     shear_range = 0.2,
  #     zoom_range = 0.2,
  #     horizontal_flip = True,
  #     fill_mode = 'nearest'
  # )

  # aug_img_data = next(datagen.flow(img_data, batch_size=1))

  features = model.predict(img_data)
  return features

def evaluate_model(model_name, X_train, X_test, y_train, y_test):
    if model_name == "XGBoost":
        model_n = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
        model_p = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
        model_k = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
        model_pH = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
    elif model_name == "RandomForest":
        model_n = RandomForestRegressor(n_estimators=100, random_state=42)
        model_p = RandomForestRegressor(n_estimators=100, random_state=42)
        model_k = RandomForestRegressor(n_estimators=100, random_state=42)
        model_pH = RandomForestRegressor(n_estimators=100, random_state=42)
    elif model_name == "GBM":
        model_n = GradientBoostingRegressor(n_estimators=100, random_state=42)
        model_p = GradientBoostingRegressor(n_estimators=100, random_state=42)
        model_k = GradientBoostingRegressor(n_estimators=100, random_state=42)
        model_pH = GradientBoostingRegressor(n_estimators=100, random_state=42)
    else:
        print("Invalid Model Name!")

    model_n.fit(X_train, y_train[:,0])
    model_p.fit(X_train, y_train[:,1])
    model_k.fit(X_train, y_train[:,2])
    model_pH.fit(X_train, y_train[:,3])

    y_pred_n = model_n.predict(X_test)
    y_pred_p = model_p.predict(X_test)
    y_pred_k = model_k.predict(X_test)
    y_pred_pH = model_pH.predict(X_test)

    mse_n = mean_squared_error(y_test[:,0], y_pred_n)
    mse_p = mean_squared_error(y_test[:,1], y_pred_p)
    mse_k = mean_squared_error(y_test[:,2], y_pred_k)
    mse_pH = mean_squared_error(y_test[:,3], y_pred_pH)

    rmse_n = np.sqrt(mse_n)
    rmse_p = np.sqrt(mse_p)
    rmse_k = np.sqrt(mse_k)
    rmse_pH = np.sqrt(mse_pH)

    mae_n = mean_absolute_error(y_test[:,0], y_pred_n)
    mae_p = mean_absolute_error(y_test[:,1], y_pred_p)
    mae_k = mean_absolute_error(y_test[:,2], y_pred_k)
    mae_pH = mean_absolute_error(y_test[:,3], y_pred_pH)

    def calculate_mpe(y_test, y_pred):
      errors = y_test - y_pred
      absolute_errors = np.abs(errors)
      relative_errors = absolute_errors / np.maximum(np.abs(y_test), 1e-6)
      mpe = np.mean(relative_errors) * 100
      return mpe


    mpe_n = calculate_mpe(y_test[:,0], y_pred_n)
    mpe_p = calculate_mpe(y_test[:,1], y_pred_p)
    mpe_k = calculate_mpe(y_test[:,2], y_pred_k)
    mpe_pH = calculate_mpe(y_test[:,3], y_pred_pH)

    r2_n = r2_score(y_test[:,0], y_pred_n)
    r2_p = r2_score(y_test[:,1], y_pred_p)
    r2_k = r2_score(y_test[:,2], y_pred_k)
    r2_pH = r2_score(y_test[:,3], y_pred_pH)

    def adjusted_r2_score(r2, n, p):
        return 1 - ((1 - r2) * ((n - 1) / (n - p - 1)))

    adj_r2_n = adjusted_r2_score(r2_n, len(y_test[:,0]), X_test.shape[1])
    adj_r2_p = adjusted_r2_score(r2_p, len(y_test[:,1]), X_test.shape[1])
    adj_r2_k = adjusted_r2_score(r2_k, len(y_test[:,2]), X_test.shape[1])
    adj_r2_pH = adjusted_r2_score(r2_pH, len(y_test[:,3]), X_test.shape[1])

    print(f"Model: {model_name}")
    print(f"RMSE (N): {rmse_n}")
    print(f"RMSE (P): {rmse_p}")
    print(f"RMSE (K): {rmse_k}")
    print(f"RMSE (pH): {rmse_pH}")
    print()
    print(f"MAE (N): {mae_n}")
    print(f"MAE (P): {mae_p}")
    print(f"MAE (K): {mae_k}")
    print(f"MAE (pH): {mae_pH}")
    print()
    print(f"MPE (N): {mpe_n}")
    print(f"MPE (P): {mpe_p}")
    print(f"MPE (K): {mpe_k}")
    print(f"MPE (pH): {mpe_pH}")
    print()
    print(f"Adjusted R-squared (N): {adj_r2_n}")
    print(f"Adjusted R-squared (P): {adj_r2_p}")
    print(f"Adjusted R-squared (K): {adj_r2_k}")
    print(f"Adjusted R-squared (pH): {adj_r2_pH}")
    print()

resnet_base_model = ResNet50(weights='imagenet',include_top=False, pooling='avg')
vgg16_base_model = VGG16(weights='imagenet', include_top=False, pooling='avg')
cnn_model = create_cnn_model((224, 224, 3))
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy')

resnet_features_list = []
vgg16_features_list = []
cnn_features_list = []

for filename in os.listdir(img_path):
  if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
    full_image_path = os.path.join(img_path, filename)

    resnet_features = extract_features(full_image_path, resnet_base_model, resnet_preprocess)
    resnet_features_list.append(resnet_features)

    vgg16_features = extract_features(full_image_path, vgg16_base_model, vgg_preprocess)
    vgg16_features_list.append(vgg16_features)

    cnn_features = extract_features(full_image_path, cnn_model,None)
    cnn_features_list.append(cnn_features)

resnet_features_array = np.vstack(resnet_features_list)
vgg16_features_array = np.vstack(vgg16_features_list)
cnn_features_array = np.vstack(cnn_features_list)

y = df[['N', 'P', 'K', 'pH']].values
print(f"Shape of y: {y.shape}")


In [ ]:
resnet_base_model.save('resnet_model.h5')
vgg16_base_model.save('vgg16_model.h5')
cnn_model.save('cnn_model.h5')

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

for features_array, model_name in [
    (resnet_features_array, "ResNet50"),
    (vgg16_features_array, "VGG16"),
    (cnn_features_array, "CNN")
]:
    X = features_array.reshape(features_array.shape[0], -1)
    print(f"Shape of {model_name} features: {X.shape}")
    print(f"Shape of y: {y.shape}")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    for reg_model in ["XGBoost", "RandomForest", "GBM"]:
        evaluate_model(reg_model, X_train, X_test, y_train, y_test)